# Traitement de l'annuaire du service public

** Où en est la parité dans les postes de chef / cheffe de la haute administration publique d'Etat  ?**

In [2]:
from bs4 import BeautifulSoup
import urllib
import re
import requests
import urllib.request
import codecs
import xml.etree.ElementTree as etree
import pandas as pd
import bs4
import os
import numpy as np
import unicodedata
from  more_itertools import unique_everseen

In [3]:
pd.options.mode.chained_assignment = None  # default='warn'

In [4]:
def retirerCaracteresSpeciaux(chaine) :
    chaine = unicodedata.normalize('NFD', chaine).encode('ascii', 'ignore').decode('utf-8')
    return(chaine)

In [5]:
def separerTitre (chaine) :
    regex_Titre = '^(.*), (.*)$'
    if re.search(regex_Titre, chaine) : 
        return(re.search(regex_Titre, chaine).groups())
    else : 
        return(['',''])

In [6]:
def separerPrenomNomTitre (chaine) :
    regex_prenomNomTitre = '^([A-Za-z \-\']*?) ([A-Z \-\']{2,})(|, *)($|[,A-Za-z -\']*)'
    if re.search(regex_prenomNomTitre, chaine) : 
        return(re.search(regex_prenomNomTitre, chaine).groups())
    else : 
        return(['', '','', ''])

In [7]:
def traiterHierarchie (chaine) :
    chaine = chaine.split('>')
    chaine = [e.lstrip().rstrip() for e in chaine]
    chaine = [e for e in chaine if e != '']
    chaine = list(unique_everseen(chaine))

    return([chaine, len(chaine)])

## Liste des services de l'annuaire

In [ ]:
racine = "https://lannuaire.service-public.fr/"
branches = ["gouvernement", "institutions-juridictions", "autorites-independantes", "ambassades", "institutions-europeennes"]
branches = [racine + branche for branche in branches]
branches_sauvegarde = []

In [ ]:
while len(branches) >0 : 
    print('Nouveau tour')
    branches_new = []
    for i, url in enumerate(branches) :
        print(i, url)
        branches_sauvegarde.append(url)
        
        html_page = urllib.request.urlopen(url)
        soup = BeautifulSoup(html_page, 'lxml')

        # Organisations
        for l0 in soup.find_all('li', itemprop = "Organization") :
            for l1 in l0.find_all('a') : 
                lien = l1['href']
                if lien not in branches_sauvegarde and lien not in branches_new: # Pour éviter les boucles à répétition
                    branches_new.append(lien)
    branches = branches_new              

In [ ]:
print(len(branches_sauvegarde))

## Téléchargement des pages

In [ ]:
# Ecriture du fichier des url à charger
with open('liens_bruts.txt', 'w') as f:
    for item in branches_sauvegarde:
        f.write("%s\n" % item)

In [ ]:
if not os.path.exists("Pages"):
    os.makedirs("Pages")

In [ ]:
for url in branches_sauvegarde :
    url_light = re.sub('^https://lannuaire.service-public.fr/', '', url)
    url_light = url_light.replace('/', '_')
    
    nom_sauvegarde = 'Pages/' + url_light + '.html'
    print(nom_sauvegarde)
    if nom_sauvegarde == ".html" :
        continue
    urllib.request.urlretrieve(url, nom_sauvegarde)

In [ ]:
# Toutes les pages ont été téléchargées
pages_chargees = [f for f in listdir('Pages/') if isfile(join('Pages/', f))]

url_light = [re.sub('^https://lannuaire.service-public.fr/', '', f) for f in branches_sauvegarde]
url_light = [f.replace('/', '_') + ".html" for f in url_light]

non_telechargees = list(set(pages_chargees) - set(url_light))
non_telechargees

## Extraction des informations

In [ ]:
annuaire = []

In [ ]:
for page in pages_chargees : 
    nom_complet = "Pages/" + page

    f = codecs.open(nom_complet, 'r', 'utf-8')
    document = BeautifulSoup(f.read(), "lxml")

    #Hiérarchie --> Nettoyage plus tard
    hierarchie = document.findAll('div', class_ = "breadcrumb")[0].text
    hierarchie = hierarchie.split('>')[1:-1]
    hierarchie = [s.replace('\xa0', '') for s in hierarchie]
    hierarchie = " > ".join(hierarchie)

    # Nom du service
    service = document.findAll('h1', id = "contentTitle")[0].text

    # Boucle sur les personnes
    personnes = document.findAll('p', itemprop = "jobTitle")
    for personne in personnes :
        try : 
            titre = personne.text
        except : 
            titre = ''
        try : 
            nom = personne.next_sibling.text
        except : 
            nom = ''
        entree = {"page" : nom_complet, "nom_service" : service, "hierarchie" : hierarchie, "titre" : titre, "personne" : nom}
        annuaire.append(entree)

In [ ]:
annuaire = pd.DataFrame(annuaire)
annuaire = annuaire.replace("\\n",' ', regex=True) 
annuaire.to_csv('annuaire.csv', sep=';', encoding='utf-8', index=False)

## Post-traitement pour nettoyer, uniformiser

In [8]:
annuaire = pd.read_csv('annuaire.csv', header=0, sep = ';')

In [9]:
annuaire['personne'] = annuaire['personne'].fillna('Non communiqué') # Si l'identité est absente
annuaire['personne_raw'] = annuaire['personne'] # Sauvegarde car plus tard, on enlèvera tous les caractères spéciaux

annuaire['personne'] = annuaire.apply(lambda row: retirerCaracteresSpeciaux(row['personne']), axis=1)
annuaire['titre'] = annuaire.apply(lambda row: retirerCaracteresSpeciaux(row['titre']), axis=1)

annuaire['prenom'] = annuaire.apply(lambda row: separerPrenomNomTitre(row['personne'])[0], axis=1).str.upper()
annuaire['nom'] = annuaire.apply(lambda row: separerPrenomNomTitre(row['personne'])[1], axis=1)
annuaire['corps'] = annuaire.apply(lambda row: separerPrenomNomTitre(row['personne'])[3], axis=1)
annuaire['corps'] = annuaire.apply(lambda row: row['corps'].replace(', ', ''), axis=1)

In [10]:
annuaire['profondeur'] = annuaire.apply(lambda row : traiterHierarchie(row['hierarchie'])[1], axis = 1) # Service placé + - haut
annuaire['hierarchie'] = annuaire.apply(lambda row : '>'.join(traiterHierarchie(row['hierarchie'])[0]), axis = 1)
annuaire['rang'] = annuaire.groupby('page').cumcount() # Ordre d'apparition sur la page

## Chargement et prétraitement de la base prénom

In [11]:
prenoms = pd.read_csv('nat2017.txt', sep = '\t')

In [12]:
prenoms['sexe'] = prenoms['sexe'].map({2 : 'female', 1: 'male'})
prenoms = prenoms.groupby(['sexe', 'preusuel'])['nombre'].sum().reset_index()
prenoms['preusuel'] = prenoms.apply(lambda row: retirerCaracteresSpeciaux(row['preusuel']), axis=1)
prenoms = prenoms.groupby(['sexe', 'preusuel'])['nombre'].sum().reset_index() # On somme à nouveau car variantes dues aux accents

prenoms = prenoms.pivot(index = 'preusuel', columns = 'sexe', values = 'nombre').reset_index()

prenoms = prenoms.fillna(0)
prenoms['total'] = prenoms['female'] + prenoms['male']
prenoms['propMale'] = prenoms['male']/prenoms['total']

prenoms.head()

sexe,preusuel,female,male,total,propMale
0,A,0.0,28.0,28.0,1.0
1,AADAM,0.0,24.0,24.0,1.0
2,AADEL,0.0,55.0,55.0,1.0
3,AADIL,0.0,177.0,177.0,1.0
4,AAKASH,0.0,25.0,25.0,1.0


In [13]:
prenoms.to_csv('prenoms_light.csv', sep=';', encoding='utf-8', index=False)
print(prenoms.shape)

(30218, 5)


## Affectation d'un genre à chaque personne

In [14]:
annuaire_genre = annuaire.merge(prenoms[['preusuel', 'propMale']], left_on='prenom', right_on='preusuel', 
                      left_index = True, how = 'left', indicator = True)
annuaire_genre = annuaire_genre.reset_index()
annuaire_genre = annuaire_genre.drop(['index'], axis=1)

annuaire_genre['genre'] = np.NaN
annuaire_genre.loc[annuaire_genre['propMale']>0.95, 'genre'] = 'M'
annuaire_genre.loc[annuaire_genre['propMale']<0.02, 'genre'] = 'F'
annuaire_genre.head()

,hierarchie,nom_service,page,personne,titre,personne_raw,prenom,nom,corps,profondeur,rang,preusuel,propMale,_merge,genre
0,Ministères>Ministère de l'Europe et des Affair...,Ambassade de France MOLDAVIE - Chisinau,Pages/ambassades_ambassade-ou-mission-diplomat...,"Pascal LE DEUNFF, ambassadeur extraordinaire e...",Ambassadeur,"Pascal LE DEUNFF, ambassadeur extraordinaire e...",PASCAL,LE DEUNFF,ambassadeur extraordinaire et plenipotentiaire,3,0,PASCAL,0.999445,both,M
1,Ministères>Ministère de l'Europe et des Affair...,Ambassade de France PAPOUASIE-NOUVELLE GUINÉE ...,Pages/ambassades_ambassade-ou-mission-diplomat...,Philippe JANVIER-KAMIYAMA,Ambassadeur,Philippe JANVIER-KAMIYAMA,PHILIPPE,JANVIER-KAMIYAMA,,3,0,PHILIPPE,0.999463,both,M
2,Ministères>Ministère de l'Europe et des Affair...,Consulat général de France ALLEMAGNE - Düsseldorf,Pages/ambassades_ambassade-ou-mission-diplomat...,"Olivia BERKELEY-CHRISTMANN, conseillere des af...",Consule generale,"Olivia BERKELEY-CHRISTMANN, conseillère des af...",OLIVIA,BERKELEY-CHRISTMANN,conseillere des affaires etrangeres,5,0,OLIVIA,0.000000,both,F
3,Ministères>Ministère de l'Europe et des Affair...,Consulat général de France LIBAN - Beyrouth,Pages/ambassades_ambassade-ou-mission-diplomat...,Karim BEN CHEIKH,Consul general,Karim BEN CHEIKH,KARIM,BEN CHEIKH,,4,0,KARIM,0.999185,both,M
4,Ministères>Ministère de l'Europe et des Affair...,Consulat général de France MAROC - Agadir,Pages/ambassades_ambassade-ou-mission-diplomat...,Dominique DOUDET,Consul general,Dominique DOUDET,DOMINIQUE,DOUDET,,4,0,DOMINIQUE,0.590075,both,NaN


In [15]:
titreF = ['directrice', 'cheffe', 'conseillere', 'presidente', 'generale', 'deleguee',
 'chargee', 'agente', 'adjointe', 'ambassadrice', 'consule', 'administratrice', 'elue']
titreM = ['ambassadeur', 'president', 'chef', 'directeur', 'controleur', 'conseiller', 'inspecteur', 
 'adjoint', 'controleur', 'delegue', 'charge', 'coordonnateur', 'general','agent', 'elu', 'commandant', 'lieutenant']

prenomsF = ['MARITSA', 'CERA', 'URWANA', 'EVGENIA', 'YASMINE-EVA', 'EDWIGE', 'HELEN']
prenomsH = ['ETIENNE-MARTIN', 'GARIN', 'THIERRY-OLIVIER', 'VALERY']

In [16]:
# Traitement des lignes indéterminées
for i, row in annuaire_genre.iterrows():
    
    if pd.isnull(row["genre"]) and row['personne'] != 'Non communique':
        mots = row["titre"].replace(',',' ').replace('-', ' ').split()
        mots = [m.lower() for m in mots]
        
        # Si la fonction est féminisée ou le prénom dans la liste des prénoms féminins ci-dessus
        if any(titre_termes in mots for titre_termes in titreF) or row['prenom'] in prenomsF :
            annuaire_genre.loc[i,'genre'] = 'F'
        # Sinon, la fonction est masculinisée ou le prénom dans la liste des prénoms masculins ci-dessus
        elif any(titre_termes in mots for titre_termes in titreM) or row['prenom'] in prenomsH :
            annuaire_genre.loc[i,'genre'] = 'M'
        # Sinon on tranche en affectant le genre le plus fréquent du nom
        else : 
            if annuaire_genre.loc[i,'propMale']> 0.5 : 
                annuaire_genre.loc[i,'genre'] = 'M'
            else :
                annuaire_genre.loc[i,'genre'] = 'F'
        

In [17]:
restant = annuaire_genre[(pd.isnull(annuaire_genre['genre'])) & (annuaire_genre['personne'] != "Non communique")]
restant.shape

(0, 15)

In [23]:
# Filtrage pour ne garder que les personnes de rang == 0, donc la profondeur est <=5 
# et dont le service est hiérarchiquement rattaché un ministère (pas une AAI ou une juridiction)
annuaire_genre = annuaire_genre[(annuaire_genre['rang'] == 0) & 
                                (annuaire_genre['profondeur'] <= 5) & 
                                (annuaire_genre['hierarchie'].str.startswith('Ministère')) &
                                (annuaire_genre['personne_raw'] != 'Non communiqué')] 

annuaire_genre = annuaire_genre.drop(['page', 'preusuel', 'propMale', '_merge', 'personne'], axis=1)
annuaire_genre.head()

In [18]:
annuaire_genre.to_csv('annuaire_genre.csv', sep=';', encoding='utf-8', index=False)